**Machine Learning**

Crear un job que entrenará
un modelo (o varios) en AutoML y desplegará el mejor modelo encontrado en un endpoint
de Azure

**Clasificcion de imagenes entre motos y vehiculos  con Computer Vision**

Dataset:

https://www.kaggle.com/datasets/utkarshsaxenadn/car-vs-bike-classification-dataset/download?datasetVersionNumber=1


In [3]:
import os
import urllib
from IPython.display import Image
from zipfile import ZipFile
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

from azure.ai.ml import MLClient
from azure.ai.ml import automl
from azure.identity import DefaultAzureCredential

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

from PIL import Image
import jsonlines

import json

def get_azure_credential(subscription_id, resource_group, workspace):
    credential = DefaultAzureCredential()
    ml_client = MLClient.from_config(credential)
    ml_client =  MLClient(credential, subscription_id, resource_group, workspace)
    return ml_client

ml_client = get_azure_credential("<subcription_id>","<resource_group>","<workspace>")

Found the config file in: /config.json


Download data from:

https://www.kaggle.com/datasets/utkarshsaxenadn/car-vs-bike-classification-dataset/download?datasetVersionNumber=1

and load zip file as "Car-Bike-Dataset.zip" under the project/data/.

In [4]:
dataset_parent_dir = "./data"

os.makedirs(dataset_parent_dir, exist_ok= True)
dataset_name = "Car-Bike-Dataset"
dataset_dir = os.path.join(dataset_parent_dir, dataset_name)
data_file = os.path.join(dataset_parent_dir, f"{dataset_name}.zip")

**Unzip Car-Bike-Dataset.zip**

In [5]:
with ZipFile(data_file, "r") as  zip:
    print("extracting files...")
    zip.extractall(path=dataset_parent_dir)
    print("done")

extracting files...
done


In [6]:
dataset_dir

'./data/Car-Bike-Dataset'

**Creando el data set de imagenes**

In [7]:
def get_azure_data(name_data, images_directory):
    my_data = Data(
        path=images_directory,
        type=AssetTypes.URI_FOLDER,
        description=name_data,
        name=name_data,
        )
    return my_data

my_data = get_azure_data("car-bike-items-images", dataset_dir)
uri_folder_data_asset = ml_client.data.create_or_update(my_data)
uri_folder_data_asset.path

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/testrc101/code/Users/ron.todo.poderoso/proyecto/data/Car-Bike-Dataset' 'https://testmlrc1011915430857.blob.core.windows.net/azureml-blobstore-68770f40-7934-479c-9806-116f9382554b/LocalUpload/4c31c224c64a6b41065d1ba5299db930/Car-Bike-Dataset' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading Car-Bike-Dataset (108.34 MBs): 100%|██████████| 108343036/108343036 [00:36<00:00, 2936219.03it/s] 




'azureml://subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/resourcegroups/rchoque/workspaces/test-ml-rc-101/datastores/workspaceblobstore/paths/LocalUpload/4c31c224c64a6b41065d1ba5299db930/Car-Bike-Dataset/'

it store the images on some place:

**'azureml://subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/resourcegroups/rchoque/workspaces/ws-test-rc-99/datastores/workspaceblobstore/paths/LocalUpload/8089963543cc1bc314894a1ef58df14d/Car-Bike-Dataset/'
**

create:
**car_bike_objects.jsonl**

In [8]:
# Define the output JSONL file path
output_jsonl_file = "car_bike_objects.jsonl"

image_dir = "./data/Car-Bike-Dataset/"

blob_storage_objects_path=uri_folder_data_asset.path

# List of classes
classes = ["Car", "Bike"]

In [9]:
# Open the JSONL file for writing
with jsonlines.open(output_jsonl_file, mode='w') as writer:
    # Loop through each class directory
    for class_name in classes:
        class_dir = os.path.join(image_dir, class_name)
        # Check if the class directory exists
        if os.path.exists(class_dir) and os.path.isdir(class_dir):
            # List all image files in the class directory
            image_files = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
            print(f'images found in {class_dir}: {len(image_files)}')
            # Loop through each image file
            for image_file in image_files:
                # Construct the full image path
                image_path = os.path.join(class_dir, image_file)
                
                # Open the image to get its details
                with Image.open(image_path) as img:
                    image_details = {
                        "format": img.format,
                        "width": img.width,
                        "height": img.height
                    }
                img_path_names = image_path.split('/')
                # Create the JSON object for this image
                image_info = {
                    "image_url": f"{blob_storage_objects_path}{img_path_names[-2]}/{img_path_names[-1]}",
                    "image_details": image_details,
                    "label": class_name
                }
                
                # Write the JSON object to the JSONL file
                writer.write(image_info)
        else:
            print(f'directory:{class_dir} does not exists')
print(f"JSONL file '{output_jsonl_file}' has been created.")

images found in ./data/Car-Bike-Dataset/Car: 2000
images found in ./data/Car-Bike-Dataset/Bike: 2000
JSONL file 'car_bike_objects.jsonl' has been created.


In [10]:
jsonl_annotations = output_jsonl_file
jsonl_annotations

'car_bike_objects.jsonl'

In [11]:
training_mltable_path = os.path.join(dataset_parent_dir, "training-mltable-folder")
validation_mltable_path = os.path.join(dataset_parent_dir, "validation-mltable-folder")

os.makedirs(training_mltable_path, exist_ok=True)
os.makedirs(validation_mltable_path, exist_ok=True)

In [14]:
train_validation_ratio = 5
training_annotations_file = os.path.join(training_mltable_path, "training_annotations.jsonl")
validation_annotations_file = os.path.join(validation_mltable_path, "validation_annotations.jsonl")

In [15]:
json_lines = None
with open(jsonl_annotations, "r") as annot_f:
    json_lines = annot_f.readlines()

In [16]:
index = 0

with open(training_annotations_file, "w") as train_f:
    with open(validation_annotations_file, "w") as validation_f:
        for json_line in json_lines:
            if index % train_validation_ratio == 0:
                validation_f.write(json_line)
            else:
                train_f.write(json_line)
            index+=1

**Create a MLtable para enviar el job de AUTO ML**

In [17]:
def create_ml_table_file(filename):
    return (
        "paths:\n"
        "  - file: ./{0}\n"
        "transformations:\n"
        "  - read_json_lines:\n"
        "        encoding: utf8\n"
        "        invalid_lines: error\n"
        "        include_path_column: false\n"
        "  - convert_column_types:\n"
        "      - columns: image_url\n"
        "        column_type: stream_info"
    ).format(filename)

def save_mltable_file(output_path, mltable_file_contents):
    file_path = os.path.join(output_path, "MLTable")
    with open(file_path, "w") as f:
        f.write(mltable_file_contents)

In [18]:
train_mltable_file_contents = create_ml_table_file(os.path.basename(training_annotations_file))
save_mltable_file(training_mltable_path, train_mltable_file_contents)

In [19]:
validation_mltable_file_contents = create_ml_table_file(os.path.basename(validation_annotations_file))
save_mltable_file(validation_mltable_path, validation_mltable_file_contents)